paket installieren:

Terminal > New Terminal

venv aktivieren:
```
.\.venv\Scripts\activate
```

paket installieren:

```
pip install pandas
``` 

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math


In [97]:
# file paths
path_motors_none_rounded_imu = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/Motors_None_Rounded/2025-01-22_17-03-38.sensors.dat"
path_motors_none_rounded_cam = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/Motors_None_Rounded/2025-01-22_17-03-38.dat"
path_imu_integral_rounded_imu = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/IMU_Integral_Rounded/2025-01-22_16-54-27.sensors.dat"
path_imu_integral_rounded_cam = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/IMU_Integral_Rounded/2025-01-22_16-54-27.dat"
path_imu_madgwick_rounded_imu = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/IMU_Madgwick_Rounded/2025-01-22_15-06-50.sensors.dat"
path_imu_madgwick_rounded_cam = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/IMU_Madgwick_Rounded/2025-01-22_15-06-50.dat"
path_imu_madgwick_square_imu = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/IMU_Madgwick_Square/2025-01-22_16-03-09.sensors.dat"
path_imu_madgwick_square_cam = "c:/Users/charl/Desktop/Uni_Leipzig/MA/Code/PlatformIO/IMU_Audit/MA_files/Recherche/Ergebnisse/IMU_Madgwick_Square/2025-01-22_16-03-09.dat"

# header für imu daten
column_names_motors = ["frame", "start_stopp", "empty0", "empty1", "empty2", "empty3", "empty4"]
column_names_madgwick = ["frame", "start_stopp", "angle", "o_w", "o_x", "o_y", "o_z"]
column_names_integral = ["frame", "start_stopp", "angle", "NaN", "gyro_x", "gyro_y", "gyro_z"]

# dateien einlesen in pd dataframes
df_path_motors_none_rounded_imu = pd.read_csv(path_motors_none_rounded_imu, sep=' ', header=None, names=column_names_motors)
df_path_motors_none_rounded_cam = pd.read_csv(path_motors_none_rounded_cam, sep=' ', header=0)
df_path_imu_integral_rounded_imu = pd.read_csv(path_imu_integral_rounded_imu, sep=' ', header=None, names=column_names_integral)
df_path_imu_integral_rounded_cam = pd.read_csv(path_imu_integral_rounded_cam, sep=' ', header=0)
df_path_imu_madgwick_rounded_imu = pd.read_csv(path_imu_madgwick_rounded_imu, sep=' ', header=None, names=column_names_madgwick)
df_path_imu_madgwick_rounded_cam = pd.read_csv(path_imu_madgwick_rounded_cam, sep=' ', header=0)
df_path_imu_madgwick_square_imu = pd.read_csv(path_imu_madgwick_square_imu, sep=' ', header=None, names=column_names_madgwick)
df_path_imu_madgwick_square_cam = pd.read_csv(path_imu_madgwick_square_cam, sep=' ', header=0)


In [98]:
# print("df_path_motors_none_rounded_imu:")
# print(df_path_motors_none_rounded_imu.head())
# print("df_path_motors_none_rounded_cam:")
# print(df_path_motors_none_rounded_cam.head())
# print("df_path_imu_integral_rounded_imu:")
# print(df_path_imu_integral_rounded_imu.head())
# print("df_path_imu_integral_rounded_cam:")
# print(df_path_imu_integral_rounded_cam.head())
# print("df_path_imu_madgwick_rounded_imu:")
# print(df_path_imu_madgwick_rounded_imu.head())
# print("df_path_imu_madgwick_rounded_cam:")
# print(df_path_imu_madgwick_rounded_cam.head())
# print("df_path_imu_madgwick_square_imu:")
# print(df_path_imu_madgwick_square_imu.head())
# print("df_path_imu_madgwick_square_cam:")
# print(df_path_imu_madgwick_square_cam.head())

In [99]:
def cyclic_mean(series, period=2 * np.pi):
    """
    Berechnet den Mittelwert einer zyklischen Serie, z. B. für Winkel oder Uhrzeiten.

    Args:
        series (pd.Series): Die Serie mit zyklischen Werten.
        period (float): Der Zyklus der Werte (z. B. 2π für Winkel, 24 für Uhrzeiten).
    
    Returns:
        float: Der zyklische Mittelwert.
    """
    # Umwandlung der Werte in Radien (wenn nicht bereits im passenden Bereich)
    radians = series * (2 * np.pi / period)  # Normierung auf [0, 2π]
    
    # Umrechnung in kartesische Koordinaten
    x = np.cos(radians)
    y = np.sin(radians)
    
    # Durchschnitt der kartesischen Koordinaten berechnen
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    
    # Zurück in zyklischen Winkel umrechnen
    mean_angle = np.arctan2(mean_y, mean_x)
    
    # Ergebnis auf den ursprünglichen Zyklus zurückskalieren
    cyclic_mean_value = (mean_angle % (2 * np.pi)) * (period / (2 * np.pi))
    
    return cyclic_mean_value


In [100]:
results = []

# für Motors_None_Rounded (avg_angle_diff: 1.5197440654558285)
# df_imu = df_path_motors_none_rounded_imu
# df_cam = df_path_motors_none_rounded_cam

# für IMU_Integral_Rounded (avg_angle_diff: 1.4782854744422966)
# df_imu = df_path_imu_integral_rounded_imu
# df_cam = df_path_imu_integral_rounded_cam

# für IMU_Madgwick_Rounded (avg_angle_diff: 1.3997784776570412)
df_imu = df_path_imu_madgwick_rounded_imu
df_cam = df_path_imu_madgwick_rounded_cam

# für IMU_Madgwick_Square (avg_angle_diff: 1.4912999356180014)
# df_imu = df_path_imu_madgwick_square_imu
# df_cam = df_path_imu_madgwick_square_cam


# Iteration über jede Zeile von df_imu
for _, row in df_imu.iterrows():
    frame_value = row['frame']
    start_stopp = row['start_stopp']
    
    # Finde die Zeile in df_cam, die diesem frame entspricht
    df_cam_before_frame = df_cam[df_cam['frame'] < frame_value]
    
    # Berechne den Durchschnitt der letzten 20 Werte in der 'angle'-Spalte
    if len(df_cam_before_frame) >= 20:
        last_20_values = df_cam_before_frame['phi'].tail(20)
        avg_angle_last_20 = cyclic_mean(last_20_values, period=2 * np.pi)
    else:
        # Falls weniger als 20 Werte vorhanden sind, den Durchschnitt der verfügbaren berechnen
        avg_angle_last_20 = cyclic_mean(df_cam_before_frame['phi'], period=2 * np.pi)
    
    # Finde die Zeile in df_cam, die dem nächsten frame entspricht
    df_cam_after_frame = df_cam[df_cam['frame'] > frame_value]

    # Berechne den Durchschnitt der nächsten 20 Werte in der 'angle'-Spalte
    if len(df_cam_after_frame) >= 20:
        next_20_values = df_cam_after_frame['phi'].head(20)
        avg_angle_next_20 = cyclic_mean(next_20_values, period=2 * np.pi)
    else:
        # Falls weniger als 20 Werte vorhanden sind, den Durchschnitt der verfügbaren berechnen
        avg_angle_next_20 = cyclic_mean(df_cam_after_frame['phi'], period=2 * np.pi)
    
    # Füge das Ergebnis in das neue DataFrame hinzu
    results.append({'frame': frame_value, 'start_stopp': start_stopp, 'avg_angle_last_20': avg_angle_last_20, 'avg_angle_next_20': avg_angle_next_20})

# Konvertiere die Ergebnisse in ein DataFrame
result_df = pd.DataFrame(results)

print(result_df)
print(last_20_values)

     frame  start_stopp  avg_angle_last_20  avg_angle_next_20
0    850.0          1.0           1.570800           1.636646
1    933.0          0.0           2.804343           3.109026
2   1058.0          1.0           3.135342           3.180947
3   1144.0          0.0           4.358588           4.676670
4   1269.0          1.0           4.690238           4.727607
..     ...          ...                ...                ...
75  8678.0          0.0           4.674846           4.982950
76  8803.0          1.0           4.986007           5.040931
77  8889.0          0.0           6.232122           0.297130
78  9014.0          1.0           0.295662           0.386457
79  9097.0          0.0           1.564355           1.832146

[80 rows x 4 columns]
8453    1.37340
8454    1.39094
8455    1.41815
8456    1.43824
8457    1.57080
8458    1.57080
8459    1.57080
8460    1.57080
8461    1.57080
8462    1.57080
8463    1.57080
8464    1.57080
8465    1.57080
8466    1.57080
8467    1

In [101]:
def cyclic_difference(value1, value2, period=2 * np.pi):
    """
    Berechnet die zyklische Differenz zwischen zwei Werten.

    Args:
        value1 (float): Erster Wert.
        value2 (float): Zweiter Wert.
        period (float): Der Zyklus der Werte (Standard: 2π).
    
    Returns:
        float: Die zyklische Differenz im Bereich [-period/2, period/2].
    """
    # Berechne die rohe Differenz
    raw_diff = value1 - value2
    
    # Bringe die Differenz in den Bereich [-period/2, period/2]
    cyclic_diff = (raw_diff + period / 2) % period - period / 2
    
    return cyclic_diff

In [102]:
# Neues DataFrame für die Ergebnisse
curves = []
curve_number = 1

# Iteration über result_df in Paaren
for i in range(len(result_df) - 1):
    if result_df.iloc[i]['start_stopp'] == 1 and result_df.iloc[i + 1]['start_stopp'] == 0:
        before_angle = result_df.iloc[i]['avg_angle_last_20']
        after_angle = result_df.iloc[i + 1]['avg_angle_next_20']
        angle_diff = cyclic_difference(after_angle, before_angle)

        # Prüfen, ob angle_diff > π und ggf. π abziehen
        # if angle_diff > math.pi:
        #     angle_diff -= 2*math.pi
        
        # Füge die Werte der aktuellen Kurve hinzu
        curves.append({
            'curve_number': curve_number,
            'before_angle': before_angle,
            'after_angle': after_angle,
            'angle_diff': angle_diff
        })
        curve_number += 1

# Konvertiere die Liste der Kurven in ein DataFrame
curves_df = pd.DataFrame(curves)

print(curves_df)

    curve_number  before_angle  after_angle  angle_diff
0              1      1.570800     3.109026    1.538226
1              2      3.135342     4.676670    1.541328
2              3      4.690238     6.257465    1.567227
3              4      6.280557     1.560906    1.563534
4              5      1.573573     3.131705    1.558132
5              6      3.143110     4.710659    1.567550
6              7      4.712385     6.281774    1.569388
7              8      0.000000     1.577460    1.577460
8              9      1.587834     3.138659    1.550825
9             10      3.143109     4.702351    1.559242
10            11      4.712385     6.268656    1.556270
11            12      6.272526     1.556368    1.567027
12            13      1.566812     3.077528    1.510715
13            14      3.098872     4.632012    1.533141
14            15      4.647046     6.169269    1.522222
15            16      6.178522     1.486426    1.591090
16            17      1.484669     3.033446    1

In [103]:
# durchschnitt für angle_diff aus curves_df berechnen
angle_diff_avg = curves_df['angle_diff'].mean()
angle_diff_std = curves_df['angle_diff'].std()
angle_diff_min = curves_df['angle_diff'].min()
angle_diff_max = curves_df['angle_diff'].max()
print("angle_diff_avg:", angle_diff_avg)
print("angle_diff_std:", angle_diff_std)
print("angle_diff_min:", angle_diff_min)
print("angle_diff_max:", angle_diff_max)

print((np.pi)/2)

angle_diff_avg: 1.5625573048503079
angle_diff_std: 0.028269316840829187
angle_diff_min: 1.4774013953564875
angle_diff_max: 1.6253780882191933
1.5707963267948966


# Motors_None_Rounded
pi/2: 1.5707963267948966 \
angle_diff_avg: 1.4932659401555364 \
angle_diff_std: 0.04038877990428226 \
angle_diff_min: 1.3719839148770987 \
angle_diff_max: 1.5812122948826817 

# IMU_Integral_Rounded
pi/2: 1.5707963267948966 \
angle_diff_avg: 1.5267675799468212 \
angle_diff_std: 0.03336129474065699 \
angle_diff_min: 1.4398230872512556 \
angle_diff_max: 1.5849497893218487 

# IMU_Madgwick_Square
pi/2: 1.5707963267948966 \
angle_diff_avg: 1.5697050276880908 \
angle_diff_std: 0.01710826117083013 \
angle_diff_min: 1.5461179065283241 \
angle_diff_max: 1.620963152947934 

# IMU_Madgwick_Rounded
pi/2: 1.5707963267948966 \
angle_diff_avg: 1.5625573048503079 \
angle_diff_std: 0.028269316840829187 \
angle_diff_min: 1.4774013953564875 \
angle_diff_max: 1.6253780882191933 